In [65]:
import sympy as sp

sMul = sp.Mul

from symqups import s as CahillGlauberS
from symqups import MP_CONFIG
from symqups.objects.scalars import (q, p, alpha, alphaD, W, mu, _Primed, 
                                     _DerivativeSymbol, hbar, StateFunction)
from symqups.objects.operators import qOp, pOp, annihilateOp, createOp, rho, Operator
from symqups.manipulations import define, qp2a, explicit, express, normal_ordered_equivalent, _deprime
from symqups.ordering import sOrdering
from symqups.quantization import s_quantize, naive_quantize
from symqups.cg import CG_transform, iCG_transform
from symqups.objects import scalars
from symqups._internal.cache import sub_cache
from symqups.star_product import Bopp, Star, _replace_diff
from symqups._internal.basic_routines import operation_routine
from symqups._internal.operator_handling import (
    separate_operator,
    separate_term_by_polynomiality,
    separate_term_oper_by_sub,
    collect_alpha_type_oper_from_monomial_by_sub,
    get_oper_sub
)
from symqups._internal.warnings import suppress_warning_repeats
from symqups._internal.grouping import UnBoppable, NotAnOperator, PhaseSpaceObject
from symqups.utils import get_random_poly
from symqups.eom import LindbladMasterEquation

from typing import Sequence

a = [alpha(i) for i in range(10)]
aop = [annihilateOp(i) for i in range(10)]
ad = [alphaD(i) for i in range(10)]
adop = [createOp(i) for i in range(10)]
x = sp.Symbol("x")
qq = [q(i) for i in range(10)]
pp = [p(i) for i in range(10)]
W = scalars.W
CahillGlauberS.val = sp.Number(0)
scalars.mu = 1
scalars.hbar = 1

CahillGlauberS

s

In [2]:
separate_term_oper_by_sub(sp.expand(iCG_transform(a[0]**2*ad[0]**3).explicit()).args[0])

[1/10, \hat{a}_{0}**2*\hat{a}^{\dagger}_{0}**3]

In [5]:
sp.simplify(normal_ordered_equivalent(iCG_transform(a[0]**2*ad[0]**3).explicit()))

3*\hat{a}^{\dagger}_{0}/2 + 3*\hat{a}^{\dagger}_{0}**2*\hat{a}_{0} + \hat{a}^{\dagger}_{0}**3*\hat{a}_{0}**2

In [ ]:
class Safe(sp.Expr, NotAnOperator):
    pass

expr = Safe(aop[0]) + aop[0]**2 + Safe(adop[1])
expr.find(lambda A: annihilateOp in (type(arg) for arg in A.args))

In [43]:
expr.find(NotAnOperator)

{Safe(\hat{a}^{\dagger}_{1}), Safe(\hat{a}_{0})}

In [81]:
only_allow_leaf_in_branch(Safe(adop[0])+Safe(adop[1]+aop[2])+a[1]*aop[1], "yeah")

ValueError: 'yeah' only accepts 'Operator' instances if they are contained within ['NotAnOperator']